In [1]:
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from datetime import datetime
import dask.dataframe as dd
import dask.array as da

In [2]:
df = dd.read_sql_table("testdaten", 'sqlite:///Kundendaten.db', "index")
df["Monats-Datum"] = df['Datum'].astype(str).str[:7]
df.head()

,Alter,Geschlecht,Job,Familienstand,Kinder,Gehalt,Angebotenes Produkt,Gekauft,Gewinn,Jahr,Monat,Tag,Datum,Anzahl,Monats-Datum
index,,,,,,,,,,,,,,,
0,35,W,Management,verheiratet,ja,89000.0,Bausparvertrag,nein,0.00,2018,1,7,2018-01-07,0,2018-01
1,40,M,Handwerk,verheiratet,nein,37000.0,Kredit,ja,0.35,2018,1,2,2018-01-02,1,2018-01
2,41,M,Administrativ,verheiratet,nein,43000.0,Kredit,ja,0.35,2018,1,16,2018-01-16,1,2018-01
3,20,M,Informatik,ledig,nein,45000.0,Kredit,ja,0.35,2018,1,3,2018-01-03,1,2018-01
4,20,M,Handwerk,ledig,nein,16000.0,Kredit,nein,0.00,2018,1,7,2018-01-07,0,2018-01


In [4]:
def Altersklassen(A):
    if A < 30:
        return 'Jung (18-29)'
    elif A <46:
        return 'Junge Erwachsene (30-45)'
    elif A<66:
        return "Alte Erwachsene (46-65)"
    else:
        return "Greise (66+)"

df['Altersklassen'] = df['Alter'].map(Altersklassen)

def Gehaltsklassen(A):
    if A < 15000:
        return 'Sehr niedrig (<15.000)'
    elif A <30000:
        return 'niedrig (15.000-30.000)'
    elif A<50000:
        return "Untere Mitte (30.000-50.000)"
    elif A<80000:
        return "Obere Mitte (50.000-80.000)"
    elif A<100000:
        return "Hoch (80.000-100.000)"
    else:
        return "Sehr hoch (>100.000)"

df['Gehaltsklassen'] = df['Gehalt'].map(Gehaltsklassen)

df.head()

,Alter,Geschlecht,Job,Familienstand,Kinder,Gehalt,Angebotenes Produkt,Gekauft,Gewinn,Jahr,Monat,Tag,Datum,Anzahl,Monats-Datum,Altersklassen,Gehaltsklassen
index,,,,,,,,,,,,,,,,,
0,35,W,Management,verheiratet,ja,89000.0,Bausparvertrag,nein,0.00,2018,1,7,2018-01-07,0,2018-01,Junge Erwachsene (30-45),Hoch (80.000-100.000)
1,40,M,Handwerk,verheiratet,nein,37000.0,Kredit,ja,0.35,2018,1,2,2018-01-02,1,2018-01,Junge Erwachsene (30-45),Untere Mitte (30.000-50.000)
2,41,M,Administrativ,verheiratet,nein,43000.0,Kredit,ja,0.35,2018,1,16,2018-01-16,1,2018-01,Junge Erwachsene (30-45),Untere Mitte (30.000-50.000)
3,20,M,Informatik,ledig,nein,45000.0,Kredit,ja,0.35,2018,1,3,2018-01-03,1,2018-01,Jung (18-29),Untere Mitte (30.000-50.000)
4,20,M,Handwerk,ledig,nein,16000.0,Kredit,nein,0.00,2018,1,7,2018-01-07,0,2018-01,Jung (18-29),niedrig (15.000-30.000)


In [7]:
df_Gewinn_pro_Monat=df.groupby(["Monats-Datum", "Angebotenes Produkt"])["Gewinn"].sum().reset_index()
df_Gewinn_pro_Monat.head()

,Monats-Datum,Angebotenes Produkt,Gewinn
0,2018-01,Altersvorsorge,515.35
1,2018-01,Bausparvertrag,704.73
2,2018-01,Depotkonto,363.48
3,2018-01,Girokonto,307.60
4,2018-01,Kredit,912.45


In [13]:
px.line(df_Gewinn_pro_Monat, x=df_Gewinn_pro_Monat["Monats-Datum"], y=df_Gewinn_pro_Monat["Gewinn"], color=df_Gewinn_pro_Monat["Angebotenes Produkt"], title="Bankprodukte ~ Gewinn")